In [1]:
# Function

In [2]:
from pathlib import Path
import optuna
import subprocess
import pickle
import numpy as np
import pandas as pd
import time
import multiprocessing

In [3]:
def random_choose_index():
    global idx, df
    return np.random.choice(idx, p=df.prob)

In [4]:
def run_command(command):
    result = subprocess.run(command, capture_output=True)
    assert (
        result.returncode == 0
    ), f"""
command:
{' '.join(command)}
returncode: {result.returncode}
stderr:
{result.stderr.decode()}
stdout:
{result.stdout.decode()}
"""
    return result

In [5]:
def run_a_instance(mean_arr, std_arr, rho_arr, indv_ants_arr, pop_size):
    global debug_mode, debug_time, acopp_dir, df, sol_dir, postfix

    df_idx = random_choose_index()
    instance_name = df.loc[df_idx].instance
    acopp_profit = df.loc[df_idx].acopp_profit

    command = [
        'python3',
        f'{acopp_dir}/run.py',
        '--acopp_dir',
        str(acopp_dir),
        '--instance_name',
        instance_name,
        '--run_only',
        '--experiment',
        # '--no_log',
        '--sol_dir',
        str(sol_dir),
        '--silent',
        '1',
        "--postfix",
        str(postfix),
        
        '--no_default',
        "--chain_flags",
        f"--lambda {pop_size} --adapt_evap --cmaes --mean_ary {mean_arr} --std_ary {std_arr} --adpt_rho {rho_arr} --indv_ants {indv_ants_arr}",
    ]

    if debug_mode:
        command += ["--time", str(debug_time)]
    
    result = run_command(command)
    stdout_log = result.stdout.decode()
    profit = int(stdout_log)
    
    gain_percent = (profit - acopp_profit) / acopp_profit * 100
    return gain_percent

In [6]:
def to_arr_flag(a_list):
    arr_flag = map(str, a_list)
    arr_flag = ":".join(arr_flag)
    return arr_flag

In [7]:
def objective(trial):
    global n_run_each_trail

    pop_size = trial.suggest_int("pop_size", 8, 15)

    indv_ants = trial.suggest_int("indv_ants", 2, 15)
    min_indv_ants = trial.suggest_int("min_indv_ants", 2, 55)
    max_indv_ants = trial.suggest_int("max_indv_ants", min_indv_ants, 55)

    rho = trial.suggest_float("rho", 0.10, 0.94)
    min_rho = trial.suggest_float("min_rho", 0.01, 0.99)
    max_rho = trial.suggest_float("max_rho", min_rho, 0.99)

    # mean + k*std <= upper_bound       <=> std <= (upper_bound - mean) / k
    # mean - k*std >= lower_bound       <=> std <= (mean - lower_bound) / k
    # std_upper_bound = min((10 - alpha_mean), (alpha_mean - 0.01))
    # std_upper_bound = max(std_upper_bound, 0.01)
    # std_upper_bound = 10 - 0.01
    # std_upper_bound = max((1 - par_a_mean), (par_a_mean - 0.01)) / 2

    alpha_mean = trial.suggest_float("alpha_mean", 0.66, 8.33)
    std_upper_bound = (8.33 - 0.66) / 2
    alpha_std = trial.suggest_float("alpha_std", 0.01, std_upper_bound)

    beta_mean = trial.suggest_float("beta_mean", alpha_mean, 9.18)
    std_upper_bound = (9.18 - 0.01) / 2
    beta_std = trial.suggest_float("beta_std", 0.01, std_upper_bound)

    par_a_mean = trial.suggest_float("par_a_mean", 0.01, 1)
    std_upper_bound = (1 - 0.01) / 2
    par_a_std = trial.suggest_float("par_a_std", 0.01, std_upper_bound)

    par_b_mean = trial.suggest_float("par_b_mean", 0.01, 1)
    std_upper_bound = (1 - 0.01) / 2
    par_b_std = trial.suggest_float("par_b_std", 0.01, std_upper_bound)

    par_c_mean = trial.suggest_float("par_c_mean", 0.01, 1)
    std_upper_bound = (1 - 0.01) / 2
    par_c_std = trial.suggest_float("par_c_std", 0.01, std_upper_bound)

    mean_arr = to_arr_flag([alpha_mean, beta_mean, par_a_mean, par_b_mean, par_c_mean])
    std_arr = to_arr_flag([alpha_std, beta_std, par_a_std, par_b_std, par_c_std])
    rho_arr = to_arr_flag([rho, min_rho, max_rho])
    indv_ants_arr = to_arr_flag([indv_ants, min_indv_ants, max_indv_ants])

    objective_value = 0
    for i in range(n_run_each_trail):
        objective_value += run_a_instance(mean_arr, std_arr, rho_arr, indv_ants_arr, pop_size) / n_run_each_trail

    return objective_value

# load dataframe

In [8]:
df = pd.read_csv("./es_ant_gain_percent.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   instance      432 non-null    object 
 1   gain_percent  432 non-null    float64
 2   prob          432 non-null    float64
 3   acopp_profit  432 non-null    float64
dtypes: float64(3), object(1)
memory usage: 13.6+ KB


In [9]:
df.describe()

,gain_percent,prob,acopp_profit
count,432.000000,432.000000,4.320000e+02
mean,-0.672013,0.002315,5.874562e+05
std,1.388989,0.000660,1.127111e+06
min,-7.434470,0.000000,2.425000e+03
25%,-1.303017,0.001963,6.249112e+04
50%,-0.368650,0.002171,1.720225e+05
75%,0.067930,0.002615,5.387015e+05
max,4.201110,0.005527,9.760000e+06


In [10]:
idx = np.arange(len(df.instance))

In [11]:
haha = np.random.choice(idx, p=df.prob)
haha, df.loc[haha].instance

(273, 'eil51_05_usw_05_03')

# Config

In [12]:
acopp_dir = Path("../")
save_path = Path("./study.pkl")
sol_dir = Path("./solutions")
postfix = str(time.time())

# total_trial = 1000
total_trial = 100
n_jobs = max(1, multiprocessing.cpu_count() // 2)
# n_run_each_trail = 5
n_run_each_trail = 3
save_each_n_trial = n_jobs

sampler = optuna.samplers.TPESampler()
idx = np.arange(len(df.instance))

debug_mode = False
# debug_mode = True
debug_time = 10
if debug_mode:
    n_jobs = 2
    n_run_each_trail = n_jobs
    total_trial = n_run_each_trail + 1
    save_each_n_trial = n_jobs

print(n_jobs)

3


# Build

In [13]:
command = [
    'python3',
    f'{acopp_dir}/run.py',
    '--acopp_dir',
    str(acopp_dir),
    '--build_only',
    '--experiment'
    ]
result = run_command(command)
print(result.stdout.decode())

$ cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -G Unix Makefiles -S.. -B../temp_build_experiment -DCMAKE_BUILD_TYPE:STRING=Release
-- Configuring done
-- Generating done
-- Build files have been written to: /mnt/c/home/vincent/data/mega/Projects/Work/Public/acoplusplus_thop_modified/src/aco++/temp_build_experiment

$ cmake --build ../temp_build_experiment -j 3
Consolidate compiler generated dependencies of target acothop
[100%] Built target acothop




# Run

In [14]:
# study = optuna.create_study(direction='maximize', sampler=sampler)
# with open(save_path, "wb") as f:
#     pickle.dump(study, f)

[I 2023-07-13 20:28:57,777] A new study created in memory with name: no-name-f5b661aa-00aa-4fb4-b86a-bc0775328492


In [15]:
with open(save_path, "rb") as f:
    study = pickle.load(f)
    
while (len(study.trials) < total_trial):
    study.optimize(objective, n_trials=save_each_n_trial, n_jobs=n_jobs)
    with open(save_path, "wb") as f:
        pickle.dump(study, f)

[I 2023-07-13 20:32:40,871] Trial 0 finished with value: -1.5431245127407178 and parameters: {'pop_size': 11, 'indv_ants': 2, 'min_indv_ants': 48, 'max_indv_ants': 56, 'rho': 0.6142195119925761, 'min_rho': 0.7363846658464949, 'max_rho': 0.9481753889629171, 'alpha_mean': 7.301432212766309, 'alpha_std': 1.03438225609394, 'beta_mean': 3.984702751352588, 'beta_std': 1.9306772298578998, 'par_a_mean': 0.38196501818207623, 'par_a_std': 0.19973113713136129, 'par_b_mean': 0.7929805148870757, 'par_b_std': 0.1876500084800389, 'par_c_mean': 0.17144947681873618, 'par_c_std': 0.07702412980970312}. Best is trial 0 with value: -1.5431245127407178.
[I 2023-07-13 20:33:12,210] Trial 2 finished with value: 0.4900116244079389 and parameters: {'pop_size': 10, 'indv_ants': 14, 'min_indv_ants': 10, 'max_indv_ants': 60, 'rho': 0.9841594461906779, 'min_rho': 0.2294586462331992, 'max_rho': 0.4691075275731784, 'alpha_mean': 4.374296316995067, 'alpha_std': 1.5865252331092223, 'beta_mean': 2.173704646337296, 'beta